In [53]:
import mne
import pandas as pd
from tqdm import tqdm

In [54]:
file_path = r'../../Data/ds004196/sub-01/ses-EEG/eeg/sub-01_ses-EEG_task-inner_eeg.bdf'
eeg_data = mne.io.read_raw_bdf(file_path, preload=True)

channels_tsv_path = r'../../Data/ds004196/sub-01/ses-EEG/eeg/sub-01_ses-EEG_task-inner_channels.tsv'
channels_info = pd.read_csv(channels_tsv_path, sep='\t')

events_tsv_path = r'../../Data/ds004196/sub-01/ses-EEG/eeg/sub-01_ses-EEG_task-inner_events.tsv'
events_info = pd.read_csv(events_tsv_path, sep='\t')

Extracting EDF parameters from /Users/omaraguilarjr/GATech/Project-InnerSpeech/Data/ds004196/sub-01/ses-EEG/eeg/sub-01_ses-EEG_task-inner_eeg.bdf...
BDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 696319  =      0.000 ...  1359.998 secs...


In [55]:
channels_info['name'] = channels_info['name'].replace('T7 (T3)', 'T7')
channels_info['name'] = channels_info['name'].replace('T8 (T4)', 'T8')
channels_info['name'] = channels_info['name'].replace('Afz', 'AFz')
channels_info['name'] = channels_info['name'].replace('Iz (inion)', 'Iz')

In [56]:
# Sort data into their channels
available_channels = set(eeg_data.ch_names).intersection(set(channels_info['name']))
excluded_channels = set(eeg_data.ch_names).symmetric_difference(set(channels_info['name']))
eeg_data.pick_channels(list(available_channels))
montage = mne.channels.make_standard_montage('biosemi64')
eeg_data.set_montage(montage, on_missing='ignore')

NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).


<RawEDF | sub-01_ses-EEG_task-inner_eeg.bdf, 64 x 696320 (1360.0 s), ~340.1 MB, data loaded>

In [57]:
# Function used to bin data
def bin_eeg_data(eeg_data, events_info):
    events_data = events_info.copy()
    events_data['onset'] = events_info['onset'] / 1000
    events_data['duration'] = events_info['duration'] / 1000

    headers = ['label', 'label_type'] + eeg_data.ch_names
    binned_data = []

    for _, event in tqdm(events_data.iterrows(), total=len(events_data), desc='EEG Event Binning'):
        if event['trial_type'] in ['wife', 'child', 'father', 'daughter']:
            label_type = 'social'
        elif event['trial_type'] in ['four', 'three', 'ten', 'six']:
            label_type = 'numeric'
        else:
            label_type = 'unknown'

        onset = event['onset']
        duration = event['duration']
        label = event['trial_type']
        
        start_sample = int(onset * eeg_data.info['sfreq'])
        stop_sample = int((onset + duration) * eeg_data.info['sfreq'])

        event_dict = {'label': label, 'label_type': label_type}
        for channel in eeg_data.info['ch_names']:
            event_dict[channel] = eeg_data.get_data(picks=channel, start=start_sample, stop=stop_sample)[0].tolist()
        
        binned_data.append(event_dict)

    binned_df = pd.DataFrame(binned_data, columns=headers)
    return binned_df

In [58]:
binned_df = bin_eeg_data(eeg_data, events_info)

EEG Event Binning: 100%|██████████| 319/319 [00:37<00:00,  8.49it/s]


In [59]:
binned_df.head

<bound method NDFrame.head of       label label_type                                                 F2  \
0       six    numeric  [0.02753246475103287, 0.02752999600559449, 0.0...   
1       six    numeric  [0.02757524592198407, 0.027573870924524718, 0....   
2      four    numeric  [0.027537620991505442, 0.027536464743641895, 0...   
3       ten    numeric  [0.027566433438267313, 0.027567839685668925, 0...   
4      four    numeric  [0.02755358971199928, 0.027551214716387672, 0....   
..      ...        ...                                                ...   
314    four    numeric  [0.028258369659743886, 0.02825721341188034, 0....   
315    four    numeric  [0.028331369524858565, 0.028328463280228568, 0...   
316   three    numeric  [0.028353369484208194, 0.028358775724218827, 0...   
317  father     social  [0.028366650709667842, 0.028370150703200737, 0...   
318  father     social  [0.028321994542181168, 0.028322275791661487, 0...   

                                             

In [60]:
binned_df.to_csv('../../Data/Processed/sub01_binned.csv', index=False)